In [177]:
import numpy as np
import pandas as pd
import datetime
import glob
#from tqdm.auto import tqdm
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from dash import Dash, html, dcc

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

import warnings
warnings.filterwarnings("ignore")

file_type = '.xlsx'

In [178]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
report_path = config['base']['report_path']
report_plot_path = config['base']['report_plot_path']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)
intraday_start_day = datetime.strptime(config['base']['intraday_start_day'], '%Y-%m-%d')
year_days = int(config['base']['year_days'])
transaction_cost = float(config['base']['transaction_cost'])


In [8]:
strategy_report_path = './strategy1\\股價突破開盤委買大單成本區.xlsx'

In [11]:
daily_report_df = pd.read_excel(strategy_report_path, sheet_name='每日報表')
daily_report_df = daily_report_df[daily_report_df['最大投入金額'] != '0'].reset_index(drop=True)
daily_report_df['Date'] = pd.to_datetime(daily_report_df['日期']) 
daily_report_df['profit'] = daily_report_df['獲利'].replace('[\$,]', '', regex=True).astype(float)
daily_report_df['max_holding_money'] = daily_report_df['最大投入金額'].replace('[\$,]', '', regex=True).astype(float)
        
max_holding_money = daily_report_df['max_holding_money'].iloc[-1]
datetime_list = [daily_report_df['Date'].iloc[0] - timedelta(days=1)] + list(daily_report_df['Date'])
nav_list = [1] + list((daily_report_df['profit'].cumsum() + max_holding_money) / max_holding_money)
nav_df = pd.DataFrame()
nav_df['nav'] = nav_list
nav_df.index = datetime_list

trading_record_df = pd.read_excel(strategy_report_path, sheet_name='交易分析')
trading_record_df = trading_record_df[['商品名稱', '進場時間', '進場方向', '進場價格', '出場時間', '出場價格', '持有區間', '交易數量', '獲利金額', '報酬率']]
trading_record_df.columns = ['ticker', 'inDate', 'direction', 'inPrice', 'outDate', 'outPrice', 'holding_days', 'trading_number', 'profit', 'returns']
trading_record_df['profit'] = trading_record_df['profit'].replace('[\$,]', '', regex=True).astype(float)
trading_record_df['returns'] = trading_record_df['profit'] / 10000
trading_record_df['inDate'] = pd.to_datetime(trading_record_df['inDate'])
trading_record_df['outDate'] = pd.to_datetime(trading_record_df['outDate'].replace('--', ''))

In [37]:
daily_report_df

,日期,獲利,總獲利,最大投入金額,商品檔數,Date,profit,max_holding_money
0,2017/11/22,"-34,807","-34,807","4,896,144",8,2017-11-22,-34807.0,4896144.0
1,2017/11/23,"-13,619","-48,426","4,896,144",6,2017-11-23,-13619.0,4896144.0
2,2017/11/24,"-17,397","-65,823","4,896,144",1,2017-11-24,-17397.0,4896144.0
3,2017/11/27,0,"-65,823","4,896,144",0,2017-11-27,0.0,4896144.0
4,2017/11/28,0,"-65,823","4,896,144",0,2017-11-28,0.0,4896144.0
...,...,...,...,...,...,...,...,...
1214,2022/11/09,"-8,109","-2,668,875","10,603,649",1,2022-11-09,-8109.0,10603649.0
1215,2022/11/10,"-11,569","-2,680,444","10,603,649",3,2022-11-10,-11569.0,10603649.0
1216,2022/11/11,"-6,328","-2,686,772","10,603,649",1,2022-11-11,-6328.0,10603649.0
1217,2022/11/14,"-3,605","-2,690,377","10,603,649",2,2022-11-14,-3605.0,10603649.0


In [38]:
trading_record_df

,ticker,inDate,direction,inPrice,outDate,outPrice,holding_days,trading_number,profit,returns
0,永豐餘(1907.TW),2017-11-22 09:00:00,買進,12.80,2017-11-22 09:40:00,12.75,41,8,-809.0,-0.0809
1,燁輝(2023.TW),2017-11-22 09:00:00,買進,11.15,2017-11-22 09:40:00,11.15,41,3,-134.0,-0.0134
2,佳世達(2352.TW),2017-11-22 09:00:00,買進,21.40,2017-11-22 09:40:00,21.35,41,2,-271.0,-0.0271
3,欣興(3037.TW),2017-11-22 09:00:00,買進,16.20,2017-11-22 09:40:00,16.20,41,1,-65.0,-0.0065
4,緯創(3231.TW),2017-11-22 09:00:00,買進,24.50,2017-11-22 09:40:00,24.05,41,38,-20790.0,-2.0790
...,...,...,...,...,...,...,...,...,...,...
5821,彩晶(6116.TW),2022-11-14 09:00:00,買進,11.95,2022-11-14 09:41:00,11.95,42,9,-430.0,-0.0430
5822,彩晶(6116.TW),2022-11-14 09:00:00,買進,11.95,2022-11-14 09:41:00,11.95,42,12,-574.0,-0.0574
5823,彩晶(6116.TW),2022-11-14 09:00:00,買進,11.95,2022-11-14 09:42:00,11.95,43,19,-908.0,-0.0908
5824,彩晶(6116.TW),2022-11-14 09:00:00,買進,11.95,2022-11-14 09:42:00,11.95,43,17,-813.0,-0.0813


In [209]:

class Strategy:
    
    def __init__(self, strategy_report_path):
        self.name = strategy_report_path[len(strategy_path):-len(file_type)]

        daily_report_df = pd.read_excel(strategy_report_path, sheet_name='每日報表')
        daily_report_df = daily_report_df[daily_report_df['最大投入金額'] != '0'].reset_index(drop=True)
        daily_report_df['Date'] = pd.to_datetime(daily_report_df['日期']) 
        daily_report_df['profit'] = daily_report_df['獲利'].replace('[\$,]', '', regex=True).astype(float)
        daily_report_df['max_holding_money'] = daily_report_df['最大投入金額'].replace('[\$,]', '', regex=True).astype(float)
                
        max_holding_money = daily_report_df['max_holding_money'].iloc[-1]
        datetime_list = [daily_report_df['Date'].iloc[0] - timedelta(days=1)] + list(daily_report_df['Date'])
        nav_list = [1] + list((daily_report_df['profit'].cumsum() + max_holding_money) / max_holding_money)
        nav_df = pd.DataFrame()
        nav_df['nav'] = nav_list
        nav_df.index = datetime_list
        

        trading_record_df = pd.read_excel(strategy_report_path, sheet_name='交易分析')
        trading_record_df = trading_record_df[['商品名稱', '進場時間', '進場方向', '進場價格', '出場時間', '出場價格', '持有區間', '交易數量', '獲利金額', '報酬率']]
        trading_record_df.columns = ['ticker', 'inDate', 'direction', 'inPrice', 'outDate', 'outPrice', 'holding_days', 'trading_number', 'profit', 'returns']
        trading_record_df['profit'] = trading_record_df['profit'].replace('[\$,]', '', regex=True).astype(float)
        trading_record_df['returns'] = trading_record_df['profit'] / 10000
        trading_record_df['inDate'] = pd.to_datetime(trading_record_df['inDate'])
        trading_record_df['outDate'] = pd.to_datetime(trading_record_df['outDate'].replace('--', ''))

        self.daily_report = daily_report_df
        self.trading_record = trading_record_df
        self.navRecords = nav_df
        self.tradingRecords = trading_record_df
        self.costs = transaction_cost / 100
        self.date_list = nav_df.index

    def DD(self, navRecords_dd = None):
        
        if navRecords_dd is None:
            navRecords_dd = self.navRecords.copy()
        
        maxDD_ratio = 0
        maxDD_num = 0
        peak_price = 0        
        
        maxDDList_ratio = []
        DDList_ratio = []
        maxDDList_num = []
        DDList_num = []
        for date in navRecords_dd.index:
            
            curPrice = navRecords_dd.loc[date, 'nav']
            
            if curPrice > peak_price:
                peak_price = curPrice
                
                maxDDList_ratio.append([date, maxDDList_ratio[-1][1] if maxDDList_ratio else 0])
                DDList_ratio.append([date, 0])
                
                maxDDList_num.append([date, maxDDList_num[-1][1] if maxDDList_num else 0])
                DDList_num.append([date, 0])

            else:
                DD_ratio = curPrice / peak_price - 1
                DD_num = curPrice - peak_price
                
                if DD_ratio < maxDD_ratio:
                    maxDD_ratio = DD_ratio
                
                if DD_num < maxDD_num:
                    maxDD_num = DD_num
                
                maxDDList_ratio.append([date, maxDD_ratio])
                DDList_ratio.append([date, DD_ratio])
                
                maxDDList_num.append([date, maxDD_num])
                DDList_num.append([date, DD_num])

        maxDDFrame_ratio = pd.DataFrame(maxDDList_ratio, columns = ['date', 'maxDD_ratio'])
        maxDDFrame_ratio.set_index('date', inplace = True)
        DDFrame_ratio = pd.DataFrame(DDList_ratio, columns = ['date', 'DD_ratio'])
        DDFrame_ratio.set_index('date', inplace = True)
        
        DDFrame_ratio = pd.concat([DDFrame_ratio, maxDDFrame_ratio], axis = 1)
        
        maxDDFrame_num = pd.DataFrame(maxDDList_num, columns = ['date', 'maxDD_num'])
        maxDDFrame_num.set_index('date', inplace = True)
        DDFrame_num = pd.DataFrame(DDList_num, columns = ['date', 'DD_num'])
        DDFrame_num.set_index('date', inplace = True)
        
        DDFrame_num = pd.concat([DDFrame_num, maxDDFrame_num], axis = 1)
        
        DDFrame = pd.concat([DDFrame_ratio, DDFrame_num], axis = 1)

        return DDFrame

    def _get_daily_holding_nums(self, tradingRecords_nums = None):
        
        if tradingRecords_nums is None:
            tradingRecords_nums = self.tradingRecords.copy()
        
        numsList = []
        for date in self.date_list:
            
            dailyHoldingRecrods = tradingRecords_nums[(tradingRecords_nums['inDate'] <= date) & (tradingRecords_nums['outDate'] >= date)]
            numsList.append([date, dailyHoldingRecrods.shape[0]])
            
        numsDF = pd.DataFrame(numsList, columns = ['date', 'HoldingNums'])
        numsDF.set_index('date', inplace = True)
        
        return numsDF
    
    def _get_daily_holding_money(self, tradingRecords = None):
        if tradingRecords is None:
            tradingRecords = self.tradingRecords.copy()
        
        moneyList = []
        for date in self.date_list:
            dailyHoldingRecrods = tradingRecords[(tradingRecords['inDate'] <= date) & (tradingRecords['outDate'] >= date)]
            moneyList.append([date, (dailyHoldingRecrods['inPrice'] * dailyHoldingRecrods['trading_number']).sum() * 1000 / 10000])
        
        moneyDF = pd.DataFrame(moneyList, columns=['date', 'HoldingMoney'])
        moneyDF.set_index('date', inplace = True)
        return moneyDF

    def _get_daily_trading_nums(self, tradingRecords_nums = None):
        
        if tradingRecords_nums is None:
            tradingRecords_nums = self.tradingRecords.copy()
        
        numsList = []
        for date in self.date_list:
            
            dailyHoldingRecrods = tradingRecords_nums[tradingRecords_nums['inDate'].dt.date == date]
            numsList.append([date, dailyHoldingRecrods.shape[0]])
    
        numsDF = pd.DataFrame(numsList, columns = ['date', 'TradingNums'])
        numsDF.set_index('date', inplace = True)
        
        return numsDF
    
    def _strategy_daily_returns(self, navFrame_ret = None):
        
        if navFrame_ret is None:
            navFrame_ret = self.navRecords.copy()
            
        lnRetFrame = np.log(navFrame_ret / navFrame_ret.shift(1)).fillna(0)
        lnRetFrame.columns = ['lnReturns']
        
        return lnRetFrame
        
    def _get_maxContinousNumber(self, navFrame, direction):
        
        if direction == "Profit":
            
            directionNum = 0
            ContinuousBeginPoint = navFrame.iloc[0]['nav']
            for i in range(1, navFrame.shape[0]):
                
                curNav = navFrame.iloc[i]['nav']
                lastNav = navFrame.iloc[i - 1]['nav']
                
                if curNav > lastNav:
                    if (curNav - ContinuousBeginPoint) > directionNum:
                        directionNum = curNav - ContinuousBeginPoint
            
                else:
                    ContinuousBeginPoint = curNav
            
            
        elif direction == "Loss":
        
            directionNum = 0
            ContinuousBeginPoint = navFrame.iloc[0]['nav']
            for i in range(1, navFrame.shape[0]):
                
                curNav = navFrame.iloc[i]['nav']
                lastNav = navFrame.iloc[i - 1]['nav']
                
                if curNav < lastNav:
                    if (curNav - ContinuousBeginPoint) < directionNum:
                        directionNum = curNav - ContinuousBeginPoint
            
                else:
                    ContinuousBeginPoint = curNav
                
        return directionNum
    
    def _get_maxPeriodNumber(self, navFrame, direction):
        
        if direction == "Profit":
            
            directionNum = 0
            upperBound = navFrame.iloc[0]['nav']
            lowerBound = navFrame.iloc[0]['nav']
            
            for i in range(1, navFrame.shape[0]):
                
                curNav = navFrame.iloc[i]['nav']
                
                if (curNav < lowerBound) | (i == navFrame.shape[0] - 1):
                    
                    if (upperBound - lowerBound) > directionNum:
                        directionNum = upperBound - lowerBound
                        
                    upperBound = curNav
                    lowerBound = curNav
                
                elif curNav > upperBound:
                    upperBound = curNav
                    
                    
        elif direction == "Loss":
        
            directionNum = 0
            upperBound = navFrame.iloc[0]['nav']
            lowerBound = navFrame.iloc[0]['nav']
            
            for i in range(1, navFrame.shape[0]):
                
                curNav = navFrame.iloc[i]['nav']
                
                if (curNav > upperBound) | (i == navFrame.shape[0] - 1):
            
                    if (lowerBound - upperBound) < directionNum:
                        directionNum = lowerBound - upperBound
                        
                    upperBound = curNav
                    lowerBound = curNav
                
                elif curNav < lowerBound:
                    lowerBound = curNav    
        
        
        return directionNum
    
    def stat(self, show = True):
        
        tradingRecords_stat = self.tradingRecords.copy()
        navRecords_stat = self.navRecords.copy()
        
        #限制資料在回測區間
        #tradingRecords_stat = tradingRecords_stat[(tradingRecords_stat['inDate'] >= self.BeginDate) & (tradingRecords_stat['outDate'] <= self.endDate)]
        #navRecords_stat = navRecords_stat[(navRecords_stat.index >= self.BeginDate) & (navRecords_stat.index <= self.endDate)]
        #navRecords_stat = navRecords_stat / navRecords_stat.iloc[0]  #rebased到回測區間第一天
        
        netNavRecords = navRecords_stat - 1
        DDFrame = self.DD(navRecords_stat)
        dailyHoldingNums = self._get_daily_holding_nums(tradingRecords_stat)
        dailyRetFrame = self._strategy_daily_returns(navRecords_stat)
        monthlyRetFrame = dailyRetFrame.resample('M').sum()
        yearlyRetFrame = dailyRetFrame.resample('Y').sum()
        
        MonthlyNavFrame = np.exp(monthlyRetFrame).cumprod()
        YearlyNavFrame = np.exp(yearlyRetFrame).cumprod()
        
        MonthlyNavFrame.columns = ['nav']
        YearlyNavFrame.columns = ['nav']        
        
        monthlyRetFrame.index = [index.strftime('%Y-%m') for index in monthlyRetFrame.index]
        yearlyRetFrame.index = [index.strftime('%Y') for index in yearlyRetFrame.index]
        
        
        #總表(筆)
        ###總報酬率(數值)
        totRet_num = navRecords_stat.iloc[-1]['nav'] - navRecords_stat.iloc[0]['nav']
        ###總報酬率(百分比)
        totRet_ratio = navRecords_stat.iloc[-1]['nav'] / navRecords_stat.iloc[0]['nav'] - 1
        ###總毛利
        grossProfit = tradingRecords_stat[tradingRecords_stat['returns'] > 0]['returns'].sum()
        ###總毛損
        grossLoss = tradingRecords_stat[tradingRecords_stat['returns'] <= 0]['returns'].sum()
        ###總淨利
        netProfit = grossProfit - abs(grossLoss)
        ###平均獲利(百分比)
        avgGrossProfit = grossProfit / tradingRecords_stat[tradingRecords_stat['returns'] > 0]['returns'].shape[0]
        ###平均虧損(百分比)
        avgGrossLoss = grossLoss / tradingRecords_stat[tradingRecords_stat['returns'] <= 0]['returns'].shape[0]
        ###平均淨利(百分比)
        avgNetProfit = netProfit / tradingRecords_stat.shape[0]
        ###獲利因子
        profitFactor = grossProfit / abs(grossLoss)
        ###平均獲利虧損比(賺賠比)
        profitLossRatio = abs(avgGrossProfit / avgGrossLoss)
        ###最大回檔(數值)
        maxDD_num = abs(DDFrame.iloc[-1]['maxDD_num'])
        ###最大回檔(百分比)
        maxDD_ratio = abs(DDFrame.iloc[-1]['maxDD_ratio'])
        ###單日歷史VaR(95%)
        VaR95_daily = np.exp(dailyRetFrame['lnReturns'].quantile(0.05)) - 1
        ###單月歷史VaR(95%)
        VaR95_monthly = np.exp(monthlyRetFrame['lnReturns'].quantile(0.05)) - 1
        ###最大持有個股數
        maxHoldingNums = dailyHoldingNums['HoldingNums'].max()
        ###CAGR & MAR
        years = (navRecords_stat.index[-1] - navRecords_stat.index[0]).days / 365
        cagr = (navRecords_stat.iloc[-1]['nav'] / navRecords_stat.iloc[0]['nav']) ** (1 / years) - 1
        mar = cagr / maxDD_ratio
        ###總交易次數
        totTradingCount = tradingRecords_stat.shape[0]
        ###獲利次數
        winTradingCount = tradingRecords_stat[tradingRecords_stat['returns'] > 0].shape[0]
        ###虧損次數
        lossTradingCount = tradingRecords_stat[tradingRecords_stat['returns'] <= 0].shape[0]
        ###勝率
        winRatio = winTradingCount / totTradingCount
        ###最大單筆交易獲利
        maxProfit_oneTrade = tradingRecords_stat['returns'].max()
        ###最大單筆交易虧損
        minProfit_oneTrade = tradingRecords_stat['returns'].min()
        ###最大連續獲利
        maxContinuousProfit = self._get_maxContinousNumber(navRecords_stat, "Profit")
        ###最大連續虧損
        maxContinuousLoss = self._get_maxContinousNumber(navRecords_stat, "Loss")
        ###最大區間獲利
        maxPeriodProfit = self._get_maxPeriodNumber(navRecords_stat, "Profit")
        ###最大區間虧損
        maxPeriodLoss = self._get_maxPeriodNumber(navRecords_stat, "Loss")

        
        #日
        ###獲利比例(日)
        winRate_daily = dailyRetFrame[dailyRetFrame['lnReturns'] > 0].shape[0] / dailyRetFrame.shape[0]
        ###交易日數
        totTradingDays = dailyRetFrame.shape[0]
        ###獲利日數
        totWinDays = dailyRetFrame[dailyRetFrame['lnReturns'] > 0].shape[0]
        ###虧損日數
        totLossDays = dailyRetFrame[dailyRetFrame['lnReturns'] <= 0].shape[0]
        ###平均毛利(百分比)
        avgGrossProfit_daily = np.exp(dailyRetFrame[dailyRetFrame['lnReturns'] > 0]['lnReturns'].mean()) - 1
        ###平均毛損(百分比)
        avgGrossLoss_daily = np.exp(dailyRetFrame[dailyRetFrame['lnReturns'] <= 0]['lnReturns'].mean()) - 1
        ###平均淨利(百分比)
        avgNetProfit_daily = np.exp(dailyRetFrame['lnReturns'].mean()) - 1
        ###最大單日獲利
        maxDailyProfit = np.exp(dailyRetFrame['lnReturns'].max()) - 1
        ###最大單日虧損
        maxDailyLoss = np.exp(dailyRetFrame['lnReturns'].min()) - 1
        ###最大連續獲利
        maxContinuousProfit_daily = maxContinuousProfit
        ###最大連續虧損
        maxContinuousLoss_daily = maxContinuousLoss
        ###最大區間獲利
        maxPeriodProfit_daily = maxPeriodProfit
        ###最大區間虧損
        maxPeriodLoss_daily = maxPeriodLoss
        
        
        #月
        ###獲利比例(月)
        winRate_monthly = monthlyRetFrame[monthlyRetFrame['lnReturns'] > 0].shape[0] / monthlyRetFrame.shape[0]
        ###交易月數
        totTradingMonths = monthlyRetFrame.shape[0]
        ###獲利月數
        totWinMonths = monthlyRetFrame[monthlyRetFrame['lnReturns'] > 0].shape[0]
        ###虧損月數
        totLossMonths = monthlyRetFrame[monthlyRetFrame['lnReturns'] <= 0].shape[0]
        ###平均毛利(百分比)
        avgGrossProfit_monthly = np.exp(monthlyRetFrame[monthlyRetFrame['lnReturns'] > 0]['lnReturns'].mean()) - 1
        ###平均毛損(百分比)
        avgGrossLoss_monthly = np.exp(monthlyRetFrame[monthlyRetFrame['lnReturns'] <= 0]['lnReturns'].mean()) - 1
        ###平均淨利(百分比)
        avgNetProfit_monthly = np.exp(monthlyRetFrame['lnReturns'].mean()) - 1
        ###最大單月獲利
        maxMonthlyProfit = np.exp(monthlyRetFrame['lnReturns'].max()) - 1
        ###最大單月虧損
        maxMonthlyLoss = np.exp(monthlyRetFrame['lnReturns'].min()) - 1
        ###最大連續獲利
        maxContinuousProfit_monthly = self._get_maxContinousNumber(MonthlyNavFrame, "Profit")
        ###最大連續虧損
        maxContinuousLoss_monthly = self._get_maxContinousNumber(MonthlyNavFrame, "Loss")
        ###最大區間獲利
        maxPeriodProfit_monthly = self._get_maxPeriodNumber(MonthlyNavFrame, "Profit")
        ###最大區間虧損
        maxPeriodLoss_monthly = self._get_maxPeriodNumber(MonthlyNavFrame, "Loss")
        
        
        #年
        ###獲利比例(年)
        winRate_yearly = yearlyRetFrame[yearlyRetFrame['lnReturns'] > 0].shape[0] / yearlyRetFrame.shape[0]
        ###交易年數
        totTradingYears = yearlyRetFrame.shape[0]
        ###獲利年數
        totWinYears = yearlyRetFrame[yearlyRetFrame['lnReturns'] > 0].shape[0]
        ###虧損年數
        totLossYears = yearlyRetFrame[yearlyRetFrame['lnReturns'] <= 0].shape[0]
        ###平均毛利(百分比)
        avgGrossProfit_yearly = np.exp(yearlyRetFrame[yearlyRetFrame['lnReturns'] > 0]['lnReturns'].mean()) - 1
        ###平均毛損(百分比)
        avgGrossLoss_yearly = np.exp(yearlyRetFrame[yearlyRetFrame['lnReturns'] <= 0]['lnReturns'].mean()) - 1
        ###平均淨利(百分比)
        avgNetProfit_yearly = np.exp(yearlyRetFrame['lnReturns'].mean()) - 1
        ###最大單年獲利
        maxYearlyProfit = np.exp(yearlyRetFrame['lnReturns'].max()) - 1
        ###最大單年虧損
        maxYearlyLoss = np.exp(yearlyRetFrame['lnReturns'].min()) - 1
        ###最大連續獲利
        maxContinuousProfit_yearly = self._get_maxContinousNumber(YearlyNavFrame, "Profit")
        ###最大連續虧損
        maxContinuousLoss_yearly = self._get_maxContinousNumber(YearlyNavFrame, "Loss")
        ###最大區間獲利
        maxPeriodProfit_yearly = self._get_maxPeriodNumber(YearlyNavFrame, "Profit")
        ###最大區間虧損
        maxPeriodLoss_yearly = self._get_maxPeriodNumber(YearlyNavFrame, "Loss")
                
        #數字統計(展示用)
        statDF_show = pd.DataFrame()
        
        #總表(筆)
        statDF_show.insert(0, "總表", ['總報酬(數值)', '總報酬(百分比)', '總毛利(萬)', '總毛損(萬)', '總淨利(萬)', '獲利因子', '平均獲利虧損比(賺賠比)', '最大回檔(數值)', '最大回檔(百分比)', '95%VaR_單日(百分比)', '95%VaR_單月(百分比)', 'CAGR', 'MAR'], allow_duplicates=True)
        statDF_show.insert(1, "總表", ['{:.2f}'.format(totRet_num), '{:.2%}'.format(totRet_ratio), '{:.2f}'.format(grossProfit), '{:.2f}'.format(grossLoss), '{:.2f}'.format(netProfit), '{:.2f}'.format(profitFactor), '{:.2f}'.format(profitLossRatio), '{:.2f}'.format(maxDD_num), '{:.2%}'.format(maxDD_ratio), '{:.2%}'.format(VaR95_daily), '{:.2%}'.format(VaR95_monthly), '{:.2%}'.format(cagr), '{:.2f}'.format(mar)], allow_duplicates=True)
        #筆
        statDF_show.insert(2, "筆", ['勝率', '總交易次數', '獲利次數', '虧損次數', '平均淨利(萬)_筆', '平均獲利(萬)_筆', '平均虧損(萬)_筆', '最大單筆交易獲利(萬)', '最大單筆交易虧損(萬)', '最大連續獲利(百分比)_筆', '最大連續虧損(百分比)_筆', '最大區間獲利(百分比)_筆', '最大區間虧損(百分比)_筆'], allow_duplicates=True)
        statDF_show.insert(3, "筆", ['{:.2%}'.format(winRatio), '{:.0f}'.format(totTradingCount), '{:.0f}'.format(winTradingCount), '{:.0f}'.format(lossTradingCount), '{:.2f}'.format(avgNetProfit), '{:.2f}'.format(avgGrossProfit), '{:.2f}'.format(avgGrossLoss), '{:.2f}'.format(maxProfit_oneTrade), '{:.2f}'.format(minProfit_oneTrade), '{:.2f}'.format(maxContinuousProfit), '{:.2f}'.format(maxContinuousLoss), '{:.2f}'.format(maxPeriodProfit), '{:.2f}'.format(maxPeriodLoss)], allow_duplicates=True)
        #日
        statDF_show.insert(4, "日", ['日獲利比率', '交易日數', '獲利日數', '虧損日數', '平均淨利(百分比)_日', '平均獲利(百分比)_日', '平均虧損(百分比)_日', '最大單日獲利', '最大單日虧損', '最大連續獲利_日', '最大連續虧損_日', '最大區間獲利_日', '最大區間虧損_日'], allow_duplicates=True)
        statDF_show.insert(5, "日", ['{:.2%}'.format(winRate_daily), '{:.0f}'.format(totTradingDays), '{:.0f}'.format(totWinDays), '{:.0f}'.format(totLossDays), '{:.2%}'.format(avgNetProfit_daily), '{:.2%}'.format(avgGrossProfit_daily), '{:.2%}'.format(avgGrossLoss_daily), '{:.2%}'.format(maxDailyProfit), '{:.2%}'.format(maxDailyLoss), '{:.2f}'.format(maxContinuousProfit_daily), '{:.2f}'.format(maxContinuousLoss_daily), '{:.2f}'.format(maxPeriodProfit_daily), '{:.2f}'.format(maxPeriodLoss_daily)], allow_duplicates=True)
        #月
        statDF_show.insert(6, "月", ['月獲利比率', '交易月數', '獲利月數', '虧損月數', '平均淨利(百分比)_月', '平均獲利(百分比)_月', '平均虧損(百分比)_月', '最大單月獲利', '最大單月虧損', '最大連續獲利_月', '最大連續虧損_月', '最大區間獲利_月', '最大區間虧損_月'], allow_duplicates=True)
        statDF_show.insert(7, "月", ['{:.2%}'.format(winRate_monthly), '{:.0f}'.format(totTradingMonths), '{:.0f}'.format(totWinMonths), '{:.0f}'.format(totLossMonths), '{:.2%}'.format(avgNetProfit_monthly), '{:.2%}'.format(avgGrossProfit_monthly), '{:.2%}'.format(avgGrossLoss_monthly), '{:.2%}'.format(maxMonthlyProfit), '{:.2%}'.format(maxMonthlyLoss), '{:.2f}'.format(maxContinuousProfit_monthly), '{:.2f}'.format(maxContinuousLoss_monthly), '{:.2f}'.format(maxPeriodProfit_monthly), '{:.2f}'.format(maxPeriodLoss_monthly)], allow_duplicates=True)
        #年
        statDF_show.insert(8, "月", ['年獲利比率', '交易年數', '獲利年數', '虧損年數', '平均淨利(百分比)_年', '平均獲利(百分比)_年', '平均虧損(百分比)_年', '最大單年獲利', '最大單年虧損', '最大連續獲利_年', '最大連續虧損_年', '最大區間獲利_年', '最大區間虧損_年'], allow_duplicates=True)
        statDF_show.insert(9, "月", ['{:.2%}'.format(winRate_yearly), '{:.0f}'.format(totTradingYears), '{:.0f}'.format(totWinYears), '{:.0f}'.format(totLossYears), '{:.2%}'.format(avgNetProfit_yearly), '{:.2%}'.format(avgGrossProfit_yearly), '{:.2%}'.format(avgGrossLoss_yearly), '{:.2%}'.format(maxYearlyProfit), '{:.2%}'.format(maxYearlyLoss), '{:.2f}'.format(maxContinuousProfit_yearly), '{:.2f}'.format(maxContinuousLoss_yearly), '{:.2f}'.format(maxPeriodProfit_yearly), '{:.2f}'.format(maxPeriodLoss_yearly)], allow_duplicates=True)
        
        statDF_show.columns = [['總表', '總表', '筆', '筆', '日', '日', '月', '月', '年', '年'], ['項目', '數值', '項目', '數值', '項目', '數值', '項目', '數值', '項目', '數值']]
        
        
        fig_stat = go.Figure(data=[go.Table(
            
            header=dict(values=list(statDF_show.columns),
                        #fill_color='paleturquoise',
                        align='center'),
            
            cells=dict(values=[statDF_show[column] for column in statDF_show.columns],
                        #fill_color='lavender',
                        align='left'))
                                
        ])
        
        fig_stat.update_layout(width=1700, height=500)
        
        if show:
            fig_stat.show()
        
        return statDF_show, fig_stat
    

    def plot_plotly(self, requiredGraphs = None):  #目前有4種圖可選['DD', 'position', 'returns hist', 'returns by freqs']，或不輸入直接取得所有

        if not requiredGraphs:
            requiredGraphs = ['DD', 'position', 'returns hist', 'returns by freqs'] 
        
        assert isinstance(requiredGraphs, list), '請確定input是list'
        
        
        tradingRecords_plot = self.tradingRecords.copy()
        navRecords_plot = self.navRecords.copy()
        
        #限制資料在回測區間
        #tradingRecords_plot = tradingRecords_plot[(tradingRecords_plot['inDate'] >= self.BeginDate) & (tradingRecords_plot['outDate'] <= self.endDate)]
        #navRecords_plot = navRecords_plot[(navRecords_plot.index >= self.BeginDate) & (navRecords_plot.index <= self.endDate)]
        #navRecords_plot = navRecords_plot / navRecords_plot.iloc[0]  #rebased到回測區間第一天
        
        netNavRecords = navRecords_plot - 1
        DDFrame = self.DD(navRecords_plot)
        dailyHoldingNums = self._get_daily_holding_nums(tradingRecords_plot)
        dailyHoldingNums = navRecords_plot.join(dailyHoldingNums)[['HoldingNums']]
        dailyHoldingMoney = self._get_daily_holding_money(tradingRecords_plot)
        dailyHoldingMoney = navRecords_plot.join(dailyHoldingMoney)[['HoldingMoney']]
        dailyTradingNums = self._get_daily_trading_nums(tradingRecords_plot)
        dailyTradingNums = navRecords_plot.join(dailyTradingNums)[['TradingNums']]
        dailyRetFrame = self._strategy_daily_returns(navRecords_plot)
        monthlyRetFrame = dailyRetFrame.resample('M').sum()
        yearlyRetFrame = dailyRetFrame.resample('Y').sum()
                
        netMonthlyNavFrame = np.exp(monthlyRetFrame).cumprod() - 1
        netYearlyNavFrame = np.exp(yearlyRetFrame).cumprod() - 1
        
        netMonthlyNavFrame.columns = ['nav']
        netYearlyNavFrame.columns = ['nav']        
        
        #monthlyRetFrame.index = [index.strftime('%Y-%m') for index in monthlyRetFrame.index]
        #yearlyRetFrame.index = [index.strftime('%Y') for index in yearlyRetFrame.index]
        
        #大盤資料
        #tickers = ["^TWII"]
        #ohlc = yf.download(tickers, start = navRecords_plot.index[0].strftime('%Y-%m-%d'), end = (navRecords_plot.index[-1] + timedelta(days = 1)).strftime('%Y-%m-%d'))
        #indexCloseFrame = ohlc[['Close']]
        #indexNavFrame = ohlc[['Close']] / ohlc[['Close']].iloc[0]
        #indexNavFrame.columns = ['nav']
        #netIndexNavFrame = indexNavFrame - 1
        
        #dailyIndexRetFrame = np.log(indexNavFrame / indexNavFrame.shift(1)).fillna(0)
        #dailyIndexRetFrame.columns = ['lnReturns']
        
        #monthlyIndexRetFrame = dailyIndexRetFrame.resample('M').sum()
        #yearlyIndexRetFrame = dailyIndexRetFrame.resample('Y').sum()
        
        #netMonthlyIndexNavFrame = np.exp(monthlyIndexRetFrame).cumprod() - 1
        #netYearlyIndexNavFrame = np.exp(yearlyIndexRetFrame).cumprod() - 1
        
        #netMonthlyIndexNavFrame.columns = ['nav']
        #netYearlyIndexNavFrame.columns = ['nav'] 
        
        
        ###開始畫圖
        #累積報酬率、DD組圖
        if 'DD' in requiredGraphs:

            fig_DD = make_subplots(rows = 3, cols = 1, subplot_titles = ('策略走勢圖', 'DD走勢圖', 'MDD走勢圖'), shared_xaxes = True)
            
            fig_DD.add_trace(go.Scatter(x = netNavRecords.index, y = netNavRecords['nav'], fill='tozeroy', name = '策略走勢'), row = 1, col = 1)
            #fig_DD.add_trace(go.Scatter(x = netIndexNavFrame.index, y = netIndexNavFrame['nav'], mode = 'lines', name = '大盤走勢'), row = 1, col = 1)
            
            fig_DD.add_trace(go.Scatter(x = DDFrame.index, y = DDFrame['DD_ratio'], fill='tozeroy', name = 'DD走勢'), row = 2, col = 1)
            fig_DD.add_trace(go.Scatter(x = DDFrame.index, y = DDFrame['maxDD_ratio'], fill='tozeroy', name = 'MDD走勢'), row = 3, col = 1)
            
            
            fig_DD.update_xaxes(
                                rangeslider = {'visible' : True, 'thickness': 0.05},
                                rangeselector = {
                                            'buttons' : list([
                                                dict(count=1, label="1m", step="month", stepmode="backward"),
                                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                                dict(step="all")
                                            ])
                                }
            )
            
            
            fig_DD.update_layout(
                                height = 900, 
                                width = 1700, #1000
                                title = {'text': '<b>策略走勢圖與回檔幅度<b>', 'x': 0.5, 'font': {'size': 30}}
                             )
            
            fig_DD.show()

        else:
            fig_DD = None
        
            
        #累積報酬率、持有部位走勢圖
        if 'position' in requiredGraphs:

            fig_pos = make_subplots(rows = 4, cols = 1, subplot_titles = ('策略走勢圖', '持有檔數走勢圖', '進場檔數走勢圖', '持倉水位走勢圖(萬)'), shared_xaxes = True)
            
            fig_pos.add_trace(go.Scatter(x = netNavRecords.index, y = netNavRecords['nav'], fill='tozeroy', name = '策略走勢'), row = 1, col = 1)
            #fig_pos.add_trace(go.Scatter(x = netIndexNavFrame.index, y = netIndexNavFrame['nav'], mode = 'lines', name = '大盤走勢'), row = 1, col = 1)
            
            fig_pos.add_trace(go.Scatter(x = dailyHoldingNums.index, y = dailyHoldingNums['HoldingNums'], name = '持有檔數走勢圖'), row = 2, col = 1)
            fig_pos.add_trace(go.Scatter(x = dailyTradingNums.index, y = dailyTradingNums['TradingNums'], name = '進場檔數走勢圖'), row = 3, col = 1)
            fig_pos.add_trace(go.Scatter(x = dailyHoldingMoney.index, y = dailyHoldingMoney['HoldingMoney'], name = '持倉水位走勢圖(萬)'), row = 4, col = 1)

            fig_pos.update_xaxes(
                                rangeslider = {'visible' : True, 'thickness': 0.05},
                                rangeselector = {
                                            'buttons' : list([
                                                dict(count=1, label="1m", step="month", stepmode="backward"),
                                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                                dict(step="all")
                                            ])
                                }
            )
            
            
            fig_pos.update_layout(
                                height = 700, 
                                width = 1700, #1700
                                title = {'text': '<b>策略走勢圖與持有部位走勢圖<b>', 'x': 0.5, 'font': {'size': 30}}
                             )
            
            fig_pos.show()
        
        else:
            fig_pos = None
        
        #報酬率分布圖
        if 'returns hist' in requiredGraphs:
            outest_prob = 0.015
            bars_num = 10
            
            returnsList = list(tradingRecords_plot['returns'])
            upperLimit = round(tradingRecords_plot['returns'].quantile(1 - outest_prob / 2), 2)
            lowerLimit = round(tradingRecords_plot['returns'].quantile(outest_prob / 2), 2)
            
            jumpRange = (upperLimit - lowerLimit) / bars_num
            
            resultDict = {}
            colorList = []
            
            resultDict[('< ' + ('{:.2f}'.format(float(lowerLimit))))] = tradingRecords_plot[tradingRecords_plot['returns'] < lowerLimit].shape[0]
    
            if lowerLimit <= 0:
                colorList.append('下跌')
            else:
                colorList.append('上漲')
    
            for i in range(bars_num):
                
                upperBound = lowerLimit + jumpRange * (i + 1)
                lowerBound = lowerLimit + jumpRange * i
                
                RangeName = ('>= ' + ('{:.2f}'.format(float(lowerBound)))) + ('  < ' + ('{:.2f}'.format(float(upperBound))))
                
                resultDict[RangeName] = tradingRecords_plot[(tradingRecords_plot['returns'] >= lowerBound) & (tradingRecords_plot['returns'] < upperBound)].shape[0]
                
                ##如果小於0的比數超過1半，那就綠色，不然就紅色
                if tradingRecords_plot[(tradingRecords_plot['returns'] >= lowerBound) & (tradingRecords_plot['returns'] < 0)].shape[0] > 0.5 * resultDict[RangeName]:
                    colorList.append('下跌')
                else:
                    colorList.append('上漲')
            
            resultDict[('>= ' + ('{:.2f}'.format(float(upperLimit))))] = tradingRecords_plot[tradingRecords_plot['returns'] >= upperLimit].shape[0]
            
            if upperLimit <= 0:
                colorList.append('下跌')
            else:
                colorList.append('上漲')
            
            resultFrame = pd.DataFrame(resultDict.items(), columns = ['range', 'count'])
            resultFrame = resultFrame.assign(color = colorList)
            resultFrame.set_index('range', inplace = True)

            fig_hist = px.bar(resultFrame, x = resultFrame.index, y= resultFrame['count'], color = 'color', color_discrete_map = {'下跌': 'green', '上漲': 'red'}, title='時間加權報酬率走勢圖')
            
            
            fig_hist.update_layout(
                                showlegend=False,
                                height = 700, 
                                width = 1700, #1700
                                title = {'text': '<b>報酬分布圖(萬)<b>', 'x': 0.5, 'font': {'size': 30}},
                                xaxis_tickangle=-60,
                             )
            
            fig_hist.show()
        
        else:
            fig_hist = None
        
        
        #日、月、年平均報酬率走勢圖
        if 'returns by freqs' in requiredGraphs:
            
            colors = {
                        '上漲': 'red',
                        '下跌': 'green'
            }
            
            dailyRetFrame = dailyRetFrame.assign(color = [('下跌' if ret < 0 else '上漲') for ret in dailyRetFrame['lnReturns']])
            monthlyRetFrame = monthlyRetFrame.assign(color = [('下跌' if ret < 0 else '上漲') for ret in monthlyRetFrame['lnReturns']])
            yearlyRetFrame = yearlyRetFrame.assign(color = [('下跌' if ret < 0 else '上漲') for ret in yearlyRetFrame['lnReturns']])
            
            #為了讓年資料的圖與時間軸對上，需要將日期設為年度第一天
            yearlyRetFrame.index = [datetime(index.year, 1, 1) for index in yearlyRetFrame.index]
            netYearlyNavFrame.index = [datetime(index.year, 1, 1) for index in netYearlyNavFrame.index]
            #netYearlyIndexNavFrame.index = [datetime(index.year, 1, 1) for index in netYearlyIndexNavFrame.index]
            
            fig_freq = make_subplots(rows = 3, cols = 2, subplot_titles = ('日平均報酬率', '時間加權報酬率_日', '月平均報酬率', '時間加權報酬率_月', '年平均報酬率', '時間加權報酬率_年'))
            
            
            #日平均報酬率
            for col in sorted(set(dailyRetFrame['color'])):
                
                tmpDF = dailyRetFrame[dailyRetFrame['color'] == col]
                fig_freq.add_trace(go.Bar(x = tmpDF.index, y = tmpDF['lnReturns'], marker_color = colors[col]), row = 1, col = 1)
            
            
            #月平均報酬率
            for col in sorted(set(monthlyRetFrame['color'])):
                
                tmpDF = monthlyRetFrame[monthlyRetFrame['color'] == col]
                fig_freq.add_trace(go.Bar(x = tmpDF.index, y = tmpDF['lnReturns'], marker_color = colors[col]), row = 2, col = 1)
            
                
            #年平均報酬率
            for col in sorted(set(yearlyRetFrame['color'])):
                
                tmpDF = yearlyRetFrame[yearlyRetFrame['color'] == col]
                fig_freq.add_trace(go.Bar(x = tmpDF.index, y = tmpDF['lnReturns'], marker_color = colors[col]), row = 3, col = 1)
            
            
            #時間加權報酬率走勢圖_日
            fig_freq.add_trace(go.Scatter(x = netNavRecords.index, y = netNavRecords['nav'], fill='tozeroy', name = '策略走勢'), row = 1, col = 2)
            #fig_freq.add_trace(go.Scatter(x = netIndexNavFrame.index, y = netIndexNavFrame['nav'], mode = 'lines', name = '大盤走勢'), row = 1, col = 2)
            
            #時間加權報酬率走勢圖_月
            fig_freq.add_trace(go.Scatter(x = netMonthlyNavFrame.index, y = netMonthlyNavFrame['nav'], fill='tozeroy', name = '策略走勢'), row = 2, col = 2)
            #fig_freq.add_trace(go.Scatter(x = netMonthlyIndexNavFrame.index, y = netMonthlyIndexNavFrame['nav'], mode = 'lines', name = '大盤走勢'), row = 2, col = 2)
            
            #時間加權報酬率走勢圖_年  
            fig_freq.add_trace(go.Scatter(x = netYearlyNavFrame.index, y = netYearlyNavFrame['nav'], fill='tozeroy', name = '策略走勢'), row = 3, col = 2)
            #fig_freq.add_trace(go.Scatter(x = netYearlyIndexNavFrame.index, y = netYearlyIndexNavFrame['nav'], mode = 'lines', name = '大盤走勢'), row = 3, col = 2)

            fig_freq.update_xaxes(
                                rangeslider = {'visible' : True, 'thickness': 0.05},
                                rangeselector = {
                                            'buttons' : list([
                                                dict(count=1, label="1m", step="month", stepmode="backward"),
                                                dict(count=6, label="6m", step="month", stepmode="backward"),
                                                dict(count=1, label="YTD", step="year", stepmode="todate"),
                                                dict(count=1, label="1y", step="year", stepmode="backward"),
                                                dict(step="all")
                                            ])
                                }
            )
            
            fig_freq.update_layout(
                                showlegend=False,
                                height = 1000, 
                                width = 1700, #1700
                                title = {'text': '<b>日、月、年平均報酬率與時間加權報酬率走勢圖<b>', 'x': 0.5, 'font': {'size': 30}}
                             )
            
            fig_freq.show()
          
        else:
            fig_freq = None
            
            
        return fig_DD, fig_pos, fig_hist, fig_freq

    def get_plotly_html(self):
        
        fig_DD, fig_pos, fig_hist, fig_freq = self.plot_plotly()
        statDF_show, fig_stat = self.stat()
        
        with open(report_plot_path + f'{self.name}.html', 'a') as f:
            f.write(fig_stat.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig_DD.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig_pos.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig_hist.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig_freq.to_html(full_html=False, include_plotlyjs='cdn'))



'./strategy\\超布林.xlsx'

In [210]:
strategy = Strategy(strategy_report_paths[0])

In [169]:
strategy.get_plotly_html()

In [268]:
from backtesting.strategy import Strategy

In [253]:
strategy.date_list[0] <= datetime(2017,11,13,9,30)

True

In [180]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
report_path = config['base']['report_path']
report_plot_path = config['base']['report_plot_path']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)
intraday_start_day = datetime.strptime(config['base']['intraday_start_day'], '%Y-%m-%d')
year_days = int(config['base']['year_days'])
transaction_cost = float(config['base']['transaction_cost'])


In [ ]:

strategy_stat = pd.DataFrame()



In [145]:

portfolio_df = pd.read_csv('./trading_day_date.txt', parse_dates=['Date'])

profit_col = []
max_holding_money_col = []
all_col = []

for strategy_report_path in strategy_report_paths:
    
    try:
        strategy = Strategy(strategy_report_path)
        name = strategy.name
        portfolio_df = pd.merge(portfolio_df, strategy.daily_report[['Date', 'profit', 'max_holding_money']], on=['Date'], how='outer')

        profit_col += [name + '_profit']
        max_holding_money_col += [name + '_max_holding_money']
        all_col += [name + '_profit', name + '_max_holding_money']
        
    except:
        continue
